# FIle rác tham khảo

1. Import thư viện

In [1]:
# import spacy
import pyodbc
import numpy as np
import regex as re
import pandas as pd

In [2]:
connect_string = "DRIVER={SQL Server};SERVER=tcp:shoprecommend.database.windows.net,1433;DATABASE=ShopRecommend;UID=ShopRecommend;PWD=Quoc2002#;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"

# Kết nối đến SQL Server
connect = pyodbc.connect(connect_string)
cursor = connect.cursor()

# Kiểm tra kết quả kết nôi
cursor.execute('''
    SELECT [ID]
        ,[ID_SK]
        ,[AccountID]
        ,[SellerID]
        ,[ProductID]
        ,[Content]
        ,[CreditRating]
        ,[IsCredited]
    FROM [dbo].[DetailComments]
    WHERE [IsCredited] is NULL
    ''')
result = cursor.fetchall()
result

[]

In [3]:
result = [(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])  for row in result]

    
# khởi tạo data frame trống
df = pd.DataFrame(result, columns=['ID', 'ID_SK', 'AccountID', 'SellerID', 'ProductID', 'Content', 'CreditRating', 'IsCredited'])
print('data shape in serrve: ',df.shape)
df

data shape in serrve:  (0, 8)


,ID,ID_SK,AccountID,SellerID,ProductID,Content,CreditRating,IsCredited


2: Thiết lập kết nối đến Serve để lấy data

In [2]:
# Code here

# Thiết lập chuỗi kết nối
# connect_string = 'DRIVER=ODBC Driver 17 for SQL Server; Server=HoangCau; Database=ShopRecommend; Trusted_Connection=yes;'
connect_string = "Driver={SQL Server};Server=HoangCau;Database=ShopRecommend;Trusted_Connection=yes;"

# Kết nối đến SQL Server
connect = pyodbc.connect(connect_string)
cursor = connect.cursor()

In [3]:
# Kiểm tra kết quả kết nôi
cursor.execute('''
SELECT TOP 100
    cmt.[AccountID], 
    cmt.[SellerID],
	cmt.[Content],
    cmt.[Rating],
	seller.[AvgRatingPoint],
	seller.[TotalFollower],
	seller.[IsOfficial],
	product.[RatingAverage],
    product.[AllTimeQuantitySold]
FROM 
    [ShopRecommend].[dbo].[DetailComments] cmt 
    JOIN [ShopRecommend].[dbo].[Seller] seller ON cmt.SellerID = seller.ID_NK 
    JOIN [ShopRecommend].[dbo].[Product] product ON cmt.ProductID = product.ID_NK 
''')
result = cursor.fetchall()
result = [(row[0], row[1], row[2], round(row[3], 5), round(row[4], 5), row[5], row[6], round(row[7], 5), row[8])  for row in result]

In [4]:

# khởi tạo data frame trống
df = pd.DataFrame(result, columns=['AccountID', 'SellerID', 'Content', 'Rating', 'SellerAvgRatingPoint', 'SellerTotalFollower', 'SellerIsOfficial', 'ProductRatingAverage', 'ProductQuantitySold'])
df.shape

(100, 9)

In [5]:
df.head()

,AccountID,SellerID,Content,Rating,SellerAvgRatingPoint,SellerTotalFollower,SellerIsOfficial,ProductRatingAverage,ProductQuantitySold
0,252,2926,"Cuốn sách rất thú vị, giúp mình ra quyết định ...",5,4.6718,500417,True,4.8,0
1,276,2926,nan,5,4.6718,500417,True,4.8,0
2,280,2926,nan,5,4.6718,500417,True,4.8,0
3,276,2926,nan,5,4.6718,500417,True,4.8,0
4,298,2926,A,5,4.6718,500417,True,4.8,0


In [6]:
df

,AccountID,SellerID,Content,Rating,SellerAvgRatingPoint,SellerTotalFollower,SellerIsOfficial,ProductRatingAverage,ProductQuantitySold
0,252,2926,"Cuốn sách rất thú vị, giúp mình ra quyết định ...",5,4.6718,500417,True,4.8,0
1,276,2926,nan,5,4.6718,500417,True,4.8,0
2,280,2926,nan,5,4.6718,500417,True,4.8,0
3,276,2926,nan,5,4.6718,500417,True,4.8,0
4,298,2926,A,5,4.6718,500417,True,4.8,0
...,...,...,...,...,...,...,...,...,...
95,908,2926,nan,5,4.6718,500417,True,4.8,0
96,911,2926,nan,5,4.6718,500417,True,4.8,0
97,914,2926,nan,5,4.6718,500417,True,4.8,0
98,918,2926,nan,5,4.6718,500417,True,4.8,0


# apply model comment

In [ ]:
import data_preprocessing
df['Content'] = df['Content'].apply(data_preprocessing.tien_xu_li)

: 

In [ ]:
df

: 

In [ ]:
from model import load_data, encode_labels, prepare_datasets, create_model, predict_label, get_path
# import model
from transformers import PhobertTokenizer, TFRobertaForSequenceClassification

: 

In [ ]:
file_path = get_path('data_processed.csv')
df_model = load_data(file_path)
# Mã hóa nhãn
df_model, label_to_encoded = encode_labels(df_model)

train_dataset, val_dataset, tokenizer = prepare_datasets(df_model)
new_model = create_model(num_labels=df_model['encoded_label'].nunique())

# # Tạo mô hình mới
# new_model = model.create_model()

# Khôi phục trọng số từ tệp checkpoint đã lưu
new_model.load_weights('./models/model_1.keras')


: 

In [ ]:
# df['Comment_predict'] = df['Content'].apply(lambda x: predict_label(new_model, tokenizer, x, label_to_encoded))

: 

In [ ]:
# df.head()

: 

In [ ]:
# Kiểm tra kết quả kết nôi
# cursor.execute('''
 
# ''')


: 

In [ ]:
input_text = 'nan'
result = predict_label(new_model, tokenizer,input_text, label_to_encoded)
result

: 

hướng dẫn sử dụng này
1/ pip install  -r requirement  để  cài  thư viện nhá
2/ tải models của hiếu về chứ nặng quá t  k  up lên đâu
3/ chạy các file 
    1-  data_preprocessing.py
    2- dataLoader.py
    3- model
4/ m đọc thấy cái input_text với cái tham số input_text  trong hàm predict là ví dụ  của 1 cái comment thì giờ thay thế bằng 1 câu comment là được
5/  có gì thì  hú t t đi ngủ đây
1:45AM 21/5

: 